In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os, math, time, random
from pathlib import Path
import torch
from torch import optim
import matplotlib.pyplot as plt
import numpy as np

# Import high-level experiment functions
from src.experiment_runner import (
    setup_experiment, load_datasets, create_models, 
    train_task_0, compute_fisher_information, 
    train_continual_learning_tasks, analyze_fisher_errors_across_timesteps,
    evaluate_fid, sample_and_visualize
)
from src.parameter_scoring import compute_param_scores
from src.ewc import EWC
from src.fisher_analysis import empirical_fisher_dense, optimal_rank1_coeff
from src.experimental_utils import analyze_fisher_approximations
import src.utils as utils

In [5]:
# Setup experiment with one function call
device, ROOT = setup_experiment(seed=123)

Device: cpu


In [6]:
# Load all datasets with one function call
datasets = load_datasets(batch_size=128)
cl_mnist_train_loaders, cl_mnist_test_loaders = datasets['cl_mnist_train'], datasets['cl_mnist_test']
cl_cifar_train_loaders, cl_cifar_test_loaders = datasets['cl_cifar_train'], datasets['cl_cifar_test']

100%|██████████| 170M/170M [00:19<00:00, 8.70MB/s] 



Extracting archive/data\cifar-10-python.tar.gz to archive/data
Files already downloaded and verified
Files already downloaded and verified
Building DataLoaders for each class in train dataset...
Building DataLoaders for each class in train dataset...


100%|██████████| 50000/50000 [00:24<00:00, 2057.28it/s]


Building DataLoaders for each class in MNIST test dataset...


100%|██████████| 10000/10000 [00:05<00:00, 1706.17it/s]



...

100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]



Extracting archive/data\MNIST\raw\t10k-labels-idx1-ubyte.gz to archive/data\MNIST\raw

Building DataLoaders for each class in train dataset...


100%|██████████| 60000/60000 [00:31<00:00, 1880.56it/s]


Building DataLoaders for each class in MNIST test dataset...


100%|██████████| 10000/10000 [00:04<00:00, 2057.76it/s]


In [7]:
# Create models and optimizers with one function call
mnist_model, cifar_model, mnist_opt, cifar_opt = create_models(device)

MNIST model parameters: 35743745
CIFAR model parameters: 35747331
CIFAR model parameters: 35747331


In [ ]:
# Train Task 0 with one function call
train_task_0(mnist_model, cl_mnist_train_loaders[0], mnist_opt, 200, ROOT, device, "mnist_model_large")

In [ ]:
# load the model
cifar_model.load_state_dict(torch.load( ROOT / "cifar_model.pth", map_location=device))
mnist_model.load_state_dict(torch.load( ROOT / "mnist_model_large.pth", map_location=device))

In [ ]:
# Using the modular compute_param_scores function
param_scores = compute_param_scores(
    mnist_model, 
    t_level=0, 
    loaders_by_class=cl_mnist_train_loaders, 
    device=device, 
    target_class=0, 
    max_samples=None
)
print(f"Parameter scores shape: {param_scores.shape}")

In [ ]:
# Compute Fisher information with one function call  
c, mu, diag, param_scores = compute_fisher_information(mnist_model, cl_mnist_train_loaders, device)

In [ ]:
# Using the modular EWC class
ewc_diag = EWC(mnist_model, "diag", diag=diag)
ewc_rank1_opt = EWC(mnist_model, "rank1_opt", mu=mu, c=c)
ewc_rank1 = EWC(mnist_model, "rank1", mu=mu, c=c)

print("EWC objects created successfully")
print(f"EWC diagonal loss example: {ewc_diag.loss().item()}")
print(f"EWC rank1_opt loss example: {ewc_rank1_opt.loss().item()}")
print(f"EWC rank1 loss example: {ewc_rank1.loss().item()}")

In [ ]:
# Train Task 1 with different EWC variants using one-liner function calls
base_state = torch.load(ROOT / "mnist_model_large.pth", map_location=device)

# Uncomment any variant you want to train:
# mnist_model.load_state_dict(base_state); train_continual_learning_tasks(mnist_model, cl_mnist_train_loaders, optim.Adam(mnist_model.parameters(), lr=2e-4), "diag", c, mu, diag, 200, ROOT, device, "diag")
# mnist_model.load_state_dict(base_state); train_continual_learning_tasks(mnist_model, cl_mnist_train_loaders, optim.Adam(mnist_model.parameters(), lr=2e-4), "rank1_opt", c, mu, diag, 200, ROOT, device, "rank1_opt")
mnist_model.load_state_dict(base_state); train_continual_learning_tasks(mnist_model, cl_mnist_train_loaders, optim.Adam(mnist_model.parameters(), lr=2e-4), "rank1", c, mu, diag, 200, ROOT, device, "rank1")

In [ ]:
print("c*:", c.item())

In [ ]:
# Using the modular compare_fisher_errors_streaming function
out = compare_fisher_errors_streaming(
    mnist_model, 500, cl_mnist_train_loaders, mu, c, device=device, target_class=0, max_samples=None
)
print("Fisher error comparison results:")
print(out)

In [ ]:
# Using the modular empirical_fisher_dense function
Fisher = empirical_fisher_dense(param_scores).to('cpu')  # (D, D)
print(f"Fisher matrix shape: {Fisher.shape}")

In [ ]:
# Using the modular optimal_rank1_coeff function
c_batch, mu_batch = optimal_rank1_coeff(param_scores, eps=1e-12, use_float64=False)
print("Optimal coefficient c*:", c_batch.item())
print(f"Mu shape: {mu_batch.shape}")

In [ ]:
# Using the modular analyze_fisher_approximations function
analysis_results = analyze_fisher_approximations(param_scores, Fisher)
print("Fisher matrix analysis:")
print(f"Diagonal error: {analysis_results['diagonal_error']}")
print(f"Rank-1 error: {analysis_results['rank1_error']}")
print(f"Optimal rank-1 error: {analysis_results['rank1_optimal_error']}")
print(f"Optimal coefficient: {analysis_results['optimal_coefficient']}")

In [ ]:
# Analyze Fisher approximation errors across timesteps with one function call
error_analysis = analyze_fisher_errors_across_timesteps(mnist_model, cl_mnist_train_loaders, device, ROOT)

In [ ]:
# Print error analysis results
print("Diagonal Errors:", error_analysis['diag_errors'])
print("Rank-1 Errors:", error_analysis['rank1_errors'])
print("Optimal Rank-1 Errors:", error_analysis['rank1_optimal_errors'])

In [ ]:
# Plot error analysis with log scale
t_levels = error_analysis['t_levels']
plt.figure(figsize=(10, 5))
plt.plot(t_levels, np.array(error_analysis['diag_errors'])*10000, label='Diagonal Error', marker='o')
plt.plot(t_levels, np.array(error_analysis['rank1_errors'])*10000, label='Rank-1 Error', marker='o')
plt.plot(t_levels, np.array(error_analysis['rank1_optimal_errors'])*10000, label='Optimal Rank-1 Error', marker='o')
plt.yscale('log')
plt.xlabel('Timestep Level')
plt.ylabel('Error Norm')
plt.title('EWC Fisher Matrix Approximation Errors')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Using the modular functions to analyze Fisher matrix
Fisher = Fisher.to('cpu')
param_scores = param_scores.to('cpu')

# Use the modular optimal_rank1_coeff function 
c_analysis, mu_analysis = optimal_rank1_coeff(param_scores, eps=1e-12, use_float64=False)

# Diagonal approximation
F_diag = torch.diag(torch.diag(Fisher))
err_diag = torch.linalg.norm(Fisher - F_diag)

# Rank-1 approximation with optimal coefficient
F_r1_score = mu_analysis.unsqueeze(1) @ mu_analysis.unsqueeze(0) * c_analysis
err_r1_score = torch.linalg.norm(Fisher - F_r1_score)

print(f"‖F-F_diag‖_F = {err_diag:.10f},  ‖F-F_r1_score‖_F = {err_r1_score:.10f}")

In [ ]:
# Empty cell for future use
pass

# test FID

In [ ]:
# Evaluate FID with one function call
fid_cifar = evaluate_fid(cifar_model, cl_cifar_test_loaders[0], device)

In [ ]:
# Load model variant and evaluate FID with one function call
mnist_model.load_state_dict(torch.load(ROOT / "mnist_model_large-task1-rank1.pth", map_location=device))
fid_mnist = evaluate_fid(mnist_model, cl_mnist_test_loaders[0], device)

In [ ]:
# Generate and visualize samples with one function call
samples = sample_and_visualize(mnist_model, device, n_samples=16)

In [ ]:
# Check available test loaders
print("Available test loader keys:", cl_mnist_test_loaders.keys())